In [ ]:
# Install HuggingFace datasets
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [ ]:
import os
import pandas as pd
import random
from datasets import load_dataset
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Connecting the Google Drive to Colab to store the datasets



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Output paths
base_dir = "/content/drive/MyDrive/amazon_data_2023"
os.makedirs(base_dir, exist_ok=True)

review_csv = os.path.join(base_dir, "amazon_review_samples.csv")
meta_csv = os.path.join(base_dir, "amazon_metadata_samples.csv")

# 1. Data Aquisition
# a) Obtain the Entire Dataset

# Sampling Amazon Reviews

This part of the code is used to collect a sample of Amazon product reviews from all categories. Since the full dataset is too large to process all at once, we take a smaller, manageable sample from each category.

1. The function **sample_reviews()** loads reviews from the Hugging Face dataset McAuley-Lab/Amazon-Reviews-2023. For each category, it randomly selects 2000 reviews and stops reading after 5 million entries to keep things efficient.

2. The parent_asin is saved into a set called collected_asins. This is a suprise tool that will help us later! (Join review data with item meta)

3. Then, each category's sample is saved to a CSV file (review_csv)and the categories that have already been processed is being tracked using a text file called completed_categories.txt (if the script crashes or needs to be rerun)

4. To be safe and avoid hitting any limits, short pauses (time.sleep) was added between processing each category (A LOT OF RUNTIME ERROR)

The output should be:

*   A combined CSV with thousands of reviews across multiple categories.
*   A list of unique parent_asin values to use when merging with metadata.
*   A progress file that tells us which categories are done.

Reference:
https://colab.research.google.com/drive/1sEQvZK94lk_YD4dc_g9m9RhtrFcut6VU?usp=sharing


In [ ]:
import time

categories = [
    "raw_review_All_Beauty", "raw_review_Amazon_Fashion", "raw_review_Appliances",
    "raw_review_Arts_Crafts_and_Sewing", "raw_review_Automotive", "raw_review_Baby_Products",
    "raw_review_Beauty_and_Personal_Care", "raw_review_Books","raw_review_CDs_and_Vinyl","raw_review_Cell_Phones_and_Accessories",
    "raw_review_Clothing_Shoes_and_Jewelry", "raw_review_Digital_Music", "raw_review_Electronics",
    "raw_review_Gift_Cards", "raw_review_Grocery_and_Gourmet_Food", "raw_review_Handmade_Products",
    "raw_review_Health_and_Household", "raw_review_Health_and_Personal_Care", "raw_review_Home_and_Kitchen",
    "raw_review_Industrial_and_Scientific", "raw_review_Kindle_Store", "raw_review_Magazine_Subscriptions", "raw_review_Movies_and_TV",
    "raw_review_Musical_Instruments", "raw_review_Office_Products", "raw_review_Patio_Lawn_and_Garden", "raw_review_Pet_Supplies",
    "raw_review_Software", "raw_review_Sports_and_Outdoors", "raw_review_Subscription_Boxes", "raw_review_Tools_and_Home_Improvement",
    "raw_review_Toys_and_Games", "raw_review_Video_Games", "raw_review_Unknown"
]

collected_asins = set() # store unique ASINs here

# Function to randomly sample reviews and collect ASINs
def sample_reviews(cat, sample_size=2000, seed=42):
    print(f"Sampling reviews from: {cat}")
    random.seed(seed)

    # a) Obtain the Entire* Dataset
    dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", name=cat, split="full", streaming=True, trust_remote_code=True)

    sample = []
    # *only a sample size of 2000 for each cat is retrieved
    for idx, example in enumerate(dataset):
        if idx < sample_size:
            sample.append(example)
        else:
            r = random.randint(0, idx)
            if r < sample_size:
                sample[r] = example
        if idx > 5_000_000:
            break

    df = pd.DataFrame(sample)

    # collect ASINs
    if 'parent_asin' in df.columns:
        collected_asins.update(df['parent_asin'].dropna().tolist())

    # review fields
    review_cols = [
    'rating',           # float
    'title',            # str
    'text',             # str
    'images',           # list
    'asin',             # str
    'parent_asin',      # str (metadata join)
    'user_id',          # str
    'timestamp',        # int (Unix time)
    'verified_purchase',# bool
    'helpful_vote'      # int
    ]

    # save it to the csv file
    df = df[[col for col in review_cols if col in df.columns]]
    df.to_csv(review_csv, mode='a', header=not os.path.exists(review_csv), index=False)
    print(f"Saved {len(df)} reviews from {cat}")
    time.sleep(2) # I am not a robot
    return True


completed_categories = set() # to track the cat that was already processed
if os.path.exists("completed_categories.txt"):
    with open("completed_categories.txt", "r") as f:
        completed_categories = set(line.strip() for line in f)


for cat in tqdm(categories, desc="Sampling reviews one-by-one"):
    if cat in completed_categories:
        print(f"Skipping {cat}, already processed.")
        continue
    try:
        sample_reviews(cat)
        with open("completed_categories.txt", "a") as f:
            f.write(cat + "\n")
        time.sleep(5) # Let me in pleasee
    except Exception as e:
        print(f"Error while processing {cat}: {e}")


print(f"\nCollected {len(collected_asins)} unique ASINs from reviews") # 58316...

Sampling reviews one-by-one:   0%|          | 0/34 [00:00<?, ?it/s]

Sampling reviews from: raw_review_All_Beauty


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

Saved 2000 reviews from raw_review_All_Beauty
Sampling reviews from: raw_review_Amazon_Fashion
Saved 2000 reviews from raw_review_Amazon_Fashion
Sampling reviews from: raw_review_Appliances
Saved 2000 reviews from raw_review_Appliances
Sampling reviews from: raw_review_Arts_Crafts_and_Sewing
Saved 2000 reviews from raw_review_Arts_Crafts_and_Sewing
Sampling reviews from: raw_review_Automotive
Saved 2000 reviews from raw_review_Automotive
Sampling reviews from: raw_review_Baby_Products
Saved 2000 reviews from raw_review_Baby_Products
Sampling reviews from: raw_review_Beauty_and_Personal_Care
Saved 2000 reviews from raw_review_Beauty_and_Personal_Care
Sampling reviews from: raw_review_Books
Saved 2000 reviews from raw_review_Books
Sampling reviews from: raw_review_CDs_and_Vinyl
Saved 2000 reviews from raw_review_CDs_and_Vinyl
Sampling reviews from: raw_review_Cell_Phones_and_Accessories
Saved 2000 reviews from raw_review_Cell_Phones_and_Accessories
Sampling reviews from: raw_review_Cloth

After sampling the reviews, we can focus on collecting the matching product metadata for the sampled reviews.

1. Similar to the above code, we keep track of which metadata categories we’ve already processed using a file called completed_meta.txt. This helps avoid downloading the same data twice if we rerun the code.

2. The function fetch_matching_metadata() loads metadata for specific categories. It streams through the category’s metadata and for each item, checks if its parent_asin matches one of the ASINs we collected from reviews. Save to the list if so.

3. While running the code, it had runtime errors and it was noted that it would reload and scan the categories again. To prevent this, once all matching metadata is found for a particular category, it’s saved to its own temporary CSV file (e.g. temp_metadata_Electronics.csv). This helps keep each category separate until we’re ready to merge everything.

4. After collecting metadata from all categories, all the temporary files are loaded and combined into one big DataFrame.Then remove duplicates based on the parent_asin and save the final cleaned and deduplicated metadata to meta_csv. Bonus: All temporary files are deleted for space management.

Output:
* A clean metadata file

In [ ]:
completed_meta = set() #to track the meta cat
if os.path.exists("completed_meta.txt"):
    with open("completed_meta.txt", "r") as f:
        completed_meta = set(line.strip() for line in f)

# Function to load metadata based on the matching ASINs of reviews
def fetch_matching_metadata(meta_cat):
    print(f"Scanning metadata: {meta_cat}")
    matches = []
    try:
        dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", name=meta_cat, split="full", streaming=True, trust_remote_code=True)
        for item in dataset:
            asin = item.get("parent_asin") # ASINs
            if asin in collected_asins:
                matches.append(item)

        if matches: # if it matches store it in its temporary cat file
            df = pd.DataFrame(matches)
            temp_filename = f"temp_metadata_{meta_cat}.csv"
            df.to_csv(temp_filename, index=False)
            print(f"Saved {len(matches)} matches to {temp_filename}")

            # Log completed category
            with open("completed_meta.txt", "a") as f:
                f.write(meta_cat + "\n")
        else:
            print(f"No matches found in {meta_cat}")
        return True

    except Exception as e:
        print(f"Error in {meta_cat}: {e}")
        return False

meta_categories = [
    "raw_meta_All_Beauty", "raw_meta_Amazon_Fashion", "raw_meta_Appliances",
    "raw_meta_Arts_Crafts_and_Sewing", "raw_meta_Automotive", "raw_meta_Baby_Products",
    "raw_meta_Beauty_and_Personal_Care", "raw_meta_Books", "raw_meta_CDs_and_Vinyl", "raw_meta_Cell_Phones_and_Accessories",
    "raw_meta_Clothing_Shoes_and_Jewelry", "raw_meta_Digital_Music", "raw_meta_Electronics",
    "raw_meta_Gift_Cards", "raw_meta_Grocery_and_Gourmet_Food", "raw_meta_Handmade_Products",
    "raw_meta_Health_and_Household", "raw_meta_Health_and_Personal_Care", "raw_meta_Home_and_Kitchen",
    "raw_meta_Industrial_and_Scientific", "raw_meta_Kindle_Store", "raw_meta_Magazine_Subscriptions", "raw_meta_Movies_and_TV",
    "raw_meta_Musical_Instruments", "raw_meta_Office_Products", "raw_meta_Patio_Lawn_and_Garden", "raw_meta_Pet_Supplies",
    "raw_meta_Software", "raw_meta_Sports_and_Outdoors", "raw_meta_Subscription_Boxes", "raw_meta_Tools_and_Home_Improvement",
    "raw_meta_Toys_and_Games", "raw_meta_Video_Games", "raw_meta_Unknown"
]

with ThreadPoolExecutor(max_workers=3) as executor: # process 3 cats at a time
    futures = [
        executor.submit(fetch_matching_metadata, cat)
        for cat in meta_categories
        if cat not in completed_meta # only if it's not found in the tracking file
    ]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching metadata"):
        future.result()

print("\n Merging all metadata files...")
temp_files = [f for f in os.listdir() if f.startswith("temp_metadata_") and f.endswith(".csv")]

# deduplication
all_dfs = []
for f in temp_files:
    df = pd.read_csv(f)
    all_dfs.append(df)

if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.drop_duplicates(subset='parent_asin', inplace=True)
    combined_df.to_csv(meta_csv, index=False) # save to the meta csv
    print(f" Final deduplicated metadata saved to: {meta_csv}")
else:
    print(" No metadata files found to merge.")

# delete temporary files for storage
for f in temp_files:
    os.remove(f)

print("All Metadata files are saved in meta_csv!")

Fetching metadata: 0it [00:00, ?it/s]


 Merging all metadata files...
 No metadata files found to merge.
All Metadata files are saved in meta_csv!


# What does the Metadata dataset look like?

In [ ]:
combined_df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Gift Cards,Tractor Supply Company Gift Card,4.9,2708.0,['Tractor Supply Company is the largest operat...,"[""Tractor Supply Company is the largest operat...",50.0,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Tractor Supply Company,"['Gift Cards', 'Gift Card Categories', 'Home I...","{""Package Dimensions"": ""5.2 x 4 x 0.01 inches;...",B01GKWEISG,NaN,NaN,NaN
1,Gift Cards,Ninety Nine Restaurants Gift Card,4.8,443.0,['Ninety Nine Restaurants are a comfortable pl...,"[""The Ninety Nine Restaurants is a comfortable...",50.0,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Ninety Nine Restaurants,"['Gift Cards', 'Gift Card Categories', 'Restau...","{""Package Dimensions"": ""5.24 x 3.94 x 0.28 inc...",B00BXLV3RO,NaN,NaN,NaN
2,Gift Cards,Amazon.com Gift Card in a Reveal (Various Desi...,4.9,80974.0,['Gift Card is affixed inside a reveal product...,"[""Amazon.com Gift Cards are the perfect way to...",20.0,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': ['Slide up to reveal card', 'Damaged...",Amazon,"['Gift Cards', 'Occasions', 'Christmas']","{""Package Dimensions"": ""4.37 x 4.21 x 0.51 inc...",B018F4M89S,NaN,NaN,NaN
3,Gift Cards,Finish Line Gift Card,4.8,540.0,"['Turn This Card Into Shoes', 'Redeemable at a...",['Finish Line Inc. is a leading athletic retai...,25.0,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Finish Line,"['Gift Cards', 'Gift Card Categories', 'Clothi...","{""Package Dimensions"": ""5.5 x 5.4 x 0.05 inche...",B00BXQXO0S,NaN,NaN,NaN
4,Gift Cards,Amazon.com Gift Card in a Greeting Card (Vario...,4.8,13831.0,['Gift Card is affixed inside a greeting card ...,"[""Amazon.com Gift Cards are the perfect way to...",NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon,"['Gift Cards', 'Gift Card Recipients', 'For Him']","{""Package Dimensions"": ""8.5 x 5.6 x 0.1 inches...",BT00CTOUNS,NaN,NaN,NaN


# Review Dataset Information

In [ ]:
review_df = pd.read_csv(review_csv)

print("Shape:", review_df.shape)

print("Number of rows:", len(review_df))

print("Columns:", review_df.columns.tolist())

review_df.head()

Shape: (68000, 10)
Number of rows: 68000
Columns: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'verified_purchase', 'helpful_vote']


,rating,title,text,images,asin,parent_asin,user_id,timestamp,verified_purchase,helpful_vote
0,5.0,Great quality foils,Not quite an exact match from the photo but I ...,[],B07WP8F672,B07WP8F672,AGEDWWZQJCZZCUVYK4SVFKLGTELQ,1595849524300,True,0
1,5.0,Good quality,Good quality thread. Several needles.,[],B07ZYTSPXN,B07ZYTSPXN,AH5SV6MBM3YNNM3P5654A63GNCVA,1589808998247,True,0
2,2.0,Not too great,Lubricating strip is too oily at first and doe...,[],B007UM7NUG,B007UM7NUG,AHJZBV4OQJNPF2Z4OIUWU4VTJUDQ,1371053208000,False,3
3,5.0,Polish that stays!,Amazing polish! Normally my polish chips the s...,[],B078924WCD,B078924WCD,AEOFMO5ZBZTG66EAGK5UEXTGWYDA,1609646481775,True,1
4,5.0,Love them,Love them,[],B08N4RMG98,B08N4RMG98,AGXPTI6RJITQCKBZDTBIFHCRMZ4A,1623054534426,True,0


# Meta Dataset Information

In [ ]:
meta_df = pd.read_csv(meta_csv)

print("Shape:", meta_df.shape)

print("Number of rows:", len(meta_df))

print("Columns:", meta_df.columns.tolist())

meta_df.head()

Shape: (56380, 16)
Number of rows: 56380
Columns: ['main_category', 'title', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images', 'videos', 'store', 'categories', 'details', 'parent_asin', 'bought_together', 'subtitle', 'author']


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Gift Cards,Tractor Supply Company Gift Card,4.9,2708.0,['Tractor Supply Company is the largest operat...,"[""Tractor Supply Company is the largest operat...",50.0,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Tractor Supply Company,"['Gift Cards', 'Gift Card Categories', 'Home I...","{""Package Dimensions"": ""5.2 x 4 x 0.01 inches;...",B01GKWEISG,NaN,NaN,NaN
1,Gift Cards,Ninety Nine Restaurants Gift Card,4.8,443.0,['Ninety Nine Restaurants are a comfortable pl...,"[""The Ninety Nine Restaurants is a comfortable...",50.0,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Ninety Nine Restaurants,"['Gift Cards', 'Gift Card Categories', 'Restau...","{""Package Dimensions"": ""5.24 x 3.94 x 0.28 inc...",B00BXLV3RO,NaN,NaN,NaN
2,Gift Cards,Amazon.com Gift Card in a Reveal (Various Desi...,4.9,80974.0,['Gift Card is affixed inside a reveal product...,"[""Amazon.com Gift Cards are the perfect way to...",20.0,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': ['Slide up to reveal card', 'Damaged...",Amazon,"['Gift Cards', 'Occasions', 'Christmas']","{""Package Dimensions"": ""4.37 x 4.21 x 0.51 inc...",B018F4M89S,NaN,NaN,NaN
3,Gift Cards,Finish Line Gift Card,4.8,540.0,"['Turn This Card Into Shoes', 'Redeemable at a...",['Finish Line Inc. is a leading athletic retai...,25.0,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Finish Line,"['Gift Cards', 'Gift Card Categories', 'Clothi...","{""Package Dimensions"": ""5.5 x 5.4 x 0.05 inche...",B00BXQXO0S,NaN,NaN,NaN
4,Gift Cards,Amazon.com Gift Card in a Greeting Card (Vario...,4.8,13831.0,['Gift Card is affixed inside a greeting card ...,"[""Amazon.com Gift Cards are the perfect way to...",NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon,"['Gift Cards', 'Gift Card Recipients', 'For Him']","{""Package Dimensions"": ""8.5 x 5.6 x 0.1 inches...",BT00CTOUNS,NaN,NaN,NaN


# 2. Data Cleaning & Preprocessing
# a) Merge on parent_asin

In [ ]:
merged_df = pd.merge(review_df, meta_df, on="parent_asin", how="inner")

print("Merged shape:", merged_df.shape)

print("Columns:", merged_df.columns.tolist())

merged_df.head()

Merged shape: (66040, 25)
Columns: ['rating', 'title_x', 'text', 'images_x', 'asin', 'parent_asin', 'user_id', 'timestamp', 'verified_purchase', 'helpful_vote', 'main_category', 'title_y', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images_y', 'videos', 'store', 'categories', 'details', 'bought_together', 'subtitle', 'author']


,rating,title_x,text,images_x,asin,parent_asin,user_id,timestamp,verified_purchase,helpful_vote,...,description,price,images_y,videos,store,categories,details,bought_together,subtitle,author
0,5.0,Great quality foils,Not quite an exact match from the photo but I ...,[],B07WP8F672,B07WP8F672,AGEDWWZQJCZZCUVYK4SVFKLGTELQ,1595849524300,True,0,...,[],NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Comdoit,[],"{""Surface Recommendation Metal"": """", """": ""Shap...",NaN,NaN,NaN
1,5.0,Good quality,Good quality thread. Several needles.,[],B07ZYTSPXN,B07ZYTSPXN,AH5SV6MBM3YNNM3P5654A63GNCVA,1589808998247,True,0,...,[],NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",AWEHIRU,[],"{""Brand"": ""Wiz Bir"", ""Included Components"": ""N...",NaN,NaN,NaN
2,2.0,Not too great,Lubricating strip is too oily at first and doe...,[],B007UM7NUG,B007UM7NUG,AHJZBV4OQJNPF2Z4OIUWU4VTJUDQ,1371053208000,False,3,...,['Gillette. The best a man can get. #1 on sens...,NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Gillette,[],"{""Brand"": ""Gillette"", ""Unit Count"": ""1 Count"",...",NaN,NaN,NaN
3,5.0,Polish that stays!,Amazing polish! Normally my polish chips the s...,[],B078924WCD,B078924WCD,AEOFMO5ZBZTG66EAGK5UEXTGWYDA,1609646481775,True,1,...,[],NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}","Starrily, Inc.",[],"{""Brand"": ""Starrily, Inc."", ""Item Form"": ""Liqu...",NaN,NaN,NaN
4,5.0,Love them,Love them,[],B08N4RMG98,B08N4RMG98,AGXPTI6RJITQCKBZDTBIFHCRMZ4A,1623054534426,True,0,...,[],NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",WXJ13,[],"{""Package Dimensions"": ""9.61 x 9.57 x 2.24 inc...",NaN,NaN,NaN


#  b) Handle Invalid / Missing Values

In [ ]:
# Drop rows where star rating is missing...
merged_df = merged_df.dropna(subset=['rating'])

# or not in [1–5].
merged_df = merged_df[merged_df['rating'].between(1, 5)]

In [ ]:
# Drop rows if text (the review body) is empty.
merged_df = merged_df.dropna(subset=['text'])

merged_df = merged_df[merged_df['text'].str.strip() != '']

In [ ]:
#If brand cannot be found in the metadata (e.g., missing in details or store), set brand = “Unknown”.

def extract_brand(details, store):
    if isinstance(details, dict) and 'brand' in details and details['brand']:
        return details['brand']
    elif pd.notna(store) and store.strip() != "":
        return store
    else:
        return "Unknown"

# Add a new column -> brand
merged_df['brand'] = merged_df.apply(lambda row: extract_brand(row.get('details', {}), row.get('store', '')), axis=1)

In [ ]:
merged_df.head()

,rating,title_x,text,images_x,asin,parent_asin,user_id,timestamp,verified_purchase,helpful_vote,...,price,images_y,videos,store,categories,details,bought_together,subtitle,author,brand
0,5.0,Great quality foils,Not quite an exact match from the photo but I ...,[],B07WP8F672,B07WP8F672,AGEDWWZQJCZZCUVYK4SVFKLGTELQ,1595849524300,True,0,...,NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Comdoit,[],"{""Surface Recommendation Metal"": """", """": ""Shap...",NaN,NaN,NaN,Comdoit
1,5.0,Good quality,Good quality thread. Several needles.,[],B07ZYTSPXN,B07ZYTSPXN,AH5SV6MBM3YNNM3P5654A63GNCVA,1589808998247,True,0,...,NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",AWEHIRU,[],"{""Brand"": ""Wiz Bir"", ""Included Components"": ""N...",NaN,NaN,NaN,AWEHIRU
2,2.0,Not too great,Lubricating strip is too oily at first and doe...,[],B007UM7NUG,B007UM7NUG,AHJZBV4OQJNPF2Z4OIUWU4VTJUDQ,1371053208000,False,3,...,NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Gillette,[],"{""Brand"": ""Gillette"", ""Unit Count"": ""1 Count"",...",NaN,NaN,NaN,Gillette
3,5.0,Polish that stays!,Amazing polish! Normally my polish chips the s...,[],B078924WCD,B078924WCD,AEOFMO5ZBZTG66EAGK5UEXTGWYDA,1609646481775,True,1,...,NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}","Starrily, Inc.",[],"{""Brand"": ""Starrily, Inc."", ""Item Form"": ""Liqu...",NaN,NaN,NaN,"Starrily, Inc."
4,5.0,Love them,Love them,[],B08N4RMG98,B08N4RMG98,AGXPTI6RJITQCKBZDTBIFHCRMZ4A,1623054534426,True,0,...,NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",WXJ13,[],"{""Package Dimensions"": ""9.61 x 9.57 x 2.24 inc...",NaN,NaN,NaN,WXJ13


In [ ]:
print(merged_df['brand'].value_counts().head(10))
# Checking the values in brand column

brand
Unknown                       1880
Amazon                        1607
Generic                        485
Hearst Magazines               323
Format: Audio CD               304
Funko                          250
Meredith Corporation           225
Allure Beauty Box              219
Amazon Basics                  190
Trusted Media Brands, Inc.     182
Name: count, dtype: int64


# c) Remove Duplicates

In [ ]:
merged_df.drop_duplicates(subset=['user_id', 'asin', 'text'], keep='first', inplace=True)

# d) Derived Columns

In [ ]:
import re

# Review Length
merged_df['review_length'] = merged_df['text'].apply(lambda x: len(re.findall(r'\b\w+\b', str(x))))

In [35]:
# Year
merged_df['year'] = pd.to_datetime(merged_df['timestamp'], unit='ms', errors='coerce').dt.year

In [36]:
print(merged_df.isna().sum()) # check Null values

rating                   0
title_x                 12
text                     0
images_x                 0
asin                     0
parent_asin              0
user_id                  0
timestamp                0
verified_purchase        0
helpful_vote             0
main_category         1480
title_y               1070
average_rating           0
rating_number           71
features              1063
description           1063
price                25238
images_y                 0
videos                   0
store                 1865
categories            1063
details                  0
bought_together      65998
subtitle             62261
author               62440
brand                    0
review_length            0
year                     0
dtype: int64


In [37]:
print("Categories:", merged_df['main_category'].nunique(), "\n", merged_df['main_category'].unique()) # check if all categories are present

Categories: 44 
 ['All Beauty' 'Premium Beauty' 'AMAZON FASHION' 'Tools & Home Improvement'
 'Amazon Home' 'Appliances' 'Camera & Photo' 'Industrial & Scientific'
 'All Electronics' 'Grocery' 'Automotive' 'Health & Personal Care'
 'Cell Phones & Accessories' nan 'Sports & Outdoors' 'Pet Supplies'
 'Office Products' 'Toys & Games' 'Musical Instruments' 'Baby'
 'Arts, Crafts & Sewing' 'Computers' 'Collectible Coins' 'Books'
 'Collectibles & Fine Art' 'Entertainment' 'Sports Collectibles'
 'Handmade' 'Home Audio & Theater' 'Car Electronics' 'GPS & Navigation'
 'Audible Audiobooks' 'Buy a Kindle' 'Digital Music' 'Movies & TV'
 'Portable Audio & Accessories' 'Amazon Devices' 'Apple Products'
 'Video Games' 'Software' 'Gift Cards' 'Magazine Subscriptions'
 'Prime Video' 'Appstore for Android' 'SUBSCRIPTION BOXES']


# e) Unified Output

In [38]:
cleaned_csv = os.path.join(base_dir, "cleaned_amazon_data.csv")
merged_df.to_csv(cleaned_csv, index=False)
print(f"Unified dataset saved to: {cleaned_csv}")

Unified dataset saved to: /content/drive/MyDrive/amazon_data_2023/cleaned_amazon_data.csv


In [39]:
# Check cleaned dataset
clean_df = pd.read_csv(cleaned_csv)

print("Shape:", clean_df.shape)

clean_df.head()

Shape: (65998, 28)


<ipython-input-39-3d3be0b86f80>:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_df = pd.read_csv(cleaned_csv)


,rating,title_x,text,images_x,asin,parent_asin,user_id,timestamp,verified_purchase,helpful_vote,...,videos,store,categories,details,bought_together,subtitle,author,brand,review_length,year
0,5.0,Great quality foils,Not quite an exact match from the photo but I ...,[],B07WP8F672,B07WP8F672,AGEDWWZQJCZZCUVYK4SVFKLGTELQ,1595849524300,True,0,...,"{'title': [], 'url': [], 'user_id': []}",Comdoit,[],"{""Surface Recommendation Metal"": """", """": ""Shap...",NaN,NaN,NaN,Comdoit,18,2020
1,5.0,Good quality,Good quality thread. Several needles.,[],B07ZYTSPXN,B07ZYTSPXN,AH5SV6MBM3YNNM3P5654A63GNCVA,1589808998247,True,0,...,"{'title': [], 'url': [], 'user_id': []}",AWEHIRU,[],"{""Brand"": ""Wiz Bir"", ""Included Components"": ""N...",NaN,NaN,NaN,AWEHIRU,5,2020
2,2.0,Not too great,Lubricating strip is too oily at first and doe...,[],B007UM7NUG,B007UM7NUG,AHJZBV4OQJNPF2Z4OIUWU4VTJUDQ,1371053208000,False,3,...,"{'title': [], 'url': [], 'user_id': []}",Gillette,[],"{""Brand"": ""Gillette"", ""Unit Count"": ""1 Count"",...",NaN,NaN,NaN,Gillette,42,2013
3,5.0,Polish that stays!,Amazing polish! Normally my polish chips the s...,[],B078924WCD,B078924WCD,AEOFMO5ZBZTG66EAGK5UEXTGWYDA,1609646481775,True,1,...,"{'title': [], 'url': [], 'user_id': []}","Starrily, Inc.",[],"{""Brand"": ""Starrily, Inc."", ""Item Form"": ""Liqu...",NaN,NaN,NaN,"Starrily, Inc.",21,2021
4,5.0,Love them,Love them,[],B08N4RMG98,B08N4RMG98,AGXPTI6RJITQCKBZDTBIFHCRMZ4A,1623054534426,True,0,...,"{'title': [], 'url': [], 'user_id': []}",WXJ13,[],"{""Package Dimensions"": ""9.61 x 9.57 x 2.24 inc...",NaN,NaN,NaN,WXJ13,2,2021


# What does the cleaned dataset look like?

In [ ]:
clean_df.head()

,rating,title_x,text,images_x,asin,parent_asin,user_id,timestamp,verified_purchase,helpful_vote,...,videos,store,categories,details,bought_together,subtitle,author,brand,review_length,year
0,5.0,Great quality foils,Not quite an exact match from the photo but I ...,[],B07WP8F672,B07WP8F672,AGEDWWZQJCZZCUVYK4SVFKLGTELQ,1595849524300,True,0,...,"{'title': [], 'url': [], 'user_id': []}",Comdoit,[],"{""Surface Recommendation Metal"": """", """": ""Shap...",NaN,NaN,NaN,Comdoit,18,2020
1,5.0,Good quality,Good quality thread. Several needles.,[],B07ZYTSPXN,B07ZYTSPXN,AH5SV6MBM3YNNM3P5654A63GNCVA,1589808998247,True,0,...,"{'title': [], 'url': [], 'user_id': []}",AWEHIRU,[],"{""Brand"": ""Wiz Bir"", ""Included Components"": ""N...",NaN,NaN,NaN,AWEHIRU,5,2020
2,2.0,Not too great,Lubricating strip is too oily at first and doe...,[],B007UM7NUG,B007UM7NUG,AHJZBV4OQJNPF2Z4OIUWU4VTJUDQ,1371053208000,False,3,...,"{'title': [], 'url': [], 'user_id': []}",Gillette,[],"{""Brand"": ""Gillette"", ""Unit Count"": ""1 Count"",...",NaN,NaN,NaN,Gillette,42,2013
3,5.0,Polish that stays!,Amazing polish! Normally my polish chips the s...,[],B078924WCD,B078924WCD,AEOFMO5ZBZTG66EAGK5UEXTGWYDA,1609646481775,True,1,...,"{'title': [], 'url': [], 'user_id': []}","Starrily, Inc.",[],"{""Brand"": ""Starrily, Inc."", ""Item Form"": ""Liqu...",NaN,NaN,NaN,"Starrily, Inc.",21,2021
4,5.0,Love them,Love them,[],B08N4RMG98,B08N4RMG98,AGXPTI6RJITQCKBZDTBIFHCRMZ4A,1623054534426,True,0,...,"{'title': [], 'url': [], 'user_id': []}",WXJ13,[],"{""Package Dimensions"": ""9.61 x 9.57 x 2.24 inc...",NaN,NaN,NaN,WXJ13,2,2021
